In [3]:
from langchain_classic.document_loaders import WebBaseLoader

In [4]:
urls = [
    "https://catalog.nau.edu/Catalog/details?plan=CSMS"
]

docs = [WebBaseLoader(url).load()[0] for url in urls]
docs

[Document(metadata={'source': 'https://catalog.nau.edu/Catalog/details?plan=CSMS', 'title': '\r\n\t\tComputer Science, Master of Science | Academic Catalog | Northern Arizona University\r\n\t', 'description': '', 'language': 'en-US'}, page_content='\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t\tComputer Science, Master of Science | Academic Catalog | Northern Arizona University\r\n\t\n\n\n\n\n\n\n\n\n\n\n2025-2026\n\n\r\n\t\t\t\t\t\t\tComputer Science, Master of Science\nSchool of Informatics, Computing and Cyber Systems\nSteve Sanghi College of Engineering\n\nComputer scientists develop complex software and computer systems that are central to contemporary science, engineering, industry, and business. The MS\xa0in Computer Science enables students to either enter the computer science workforce or continue on to a doctoral program of study.\xa0Individual and team-based assignments will enable students to build mastery of important computer science skills and their practical a

In [30]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200, separators = ["\n\n", "\n", " ", ""])

chunks = text_splitter.split_documents(docs)
chunks

[Document(metadata={'source': 'https://catalog.nau.edu/Catalog/details?plan=CSMS', 'title': '\r\n\t\tComputer Science, Master of Science | Academic Catalog | Northern Arizona University\r\n\t', 'description': '', 'language': 'en-US'}, page_content='Computer Science, Master of Science | Academic Catalog | Northern Arizona University\r\n\t\n\n\n\n\n\n\n\n\n\n\n2025-2026\n\n\r\n\t\t\t\t\t\t\tComputer Science, Master of Science\nSchool of Informatics, Computing and Cyber Systems\nSteve Sanghi College of Engineering'),
 Document(metadata={'source': 'https://catalog.nau.edu/Catalog/details?plan=CSMS', 'title': '\r\n\t\tComputer Science, Master of Science | Academic Catalog | Northern Arizona University\r\n\t', 'description': '', 'language': 'en-US'}, page_content='Computer scientists develop complex software and computer systems that are central to contemporary science, engineering, industry, and business. The MS\xa0in Computer Science enables students to either enter the computer science wo

In [58]:
from langchain_classic.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [32]:
from langchain_classic.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    chunks, embeddings
)
vectorstore.save_local("vectorstore")

In [8]:
from langchain_ollama.llms import OllamaLLM
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.schema.runnable import RunnablePassthrough
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

In [33]:
from langchain_community.vectorstores import FAISS
vectorstore_ = FAISS.load_local("vectorstore", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore_.as_retriever(kwargs={"k":5}) #fetches relevant documents and returns the top k chunks based on similarity

In [34]:
query = "What are the core requirements for the NAU Computer Science MS program?"
retrieved_docs = retriever.invoke(query)
context = ' '.join([doc.page_content for doc in retrieved_docs])
context

"Additional Admission Requirements\n\n\n\n\n\nIndividual program admission requirements over and above admission to NAU are required.\n\n\n\nA Bachelor degree in Computer Science or a related field.\n\n\n\nComputer Science Milestone. Students must pass the Initial Skills Inventory Exam, or complete CS 500 and CS 501 before enrolling in graduate-level CS courses. See the department website for details.\n\n\n\nEssay/Letter of Intent/Personal Statement*\n\n\n\nRecommendation(s)/Reference(s)*\n\n\n\n*See the application for details.\n\nMaster's Requirements\n\n\n\n\n\nThis Master's degree requires 30 units distributed as follows: Computer Science Coursework:\xa03 units Non-Thesis or Thesis Option: 27 units\t Non-Thesis Option Thesis Option Thesis Option with Research  Take the following 30 units:\n\n\n\nStudents completing a non-thesis, coursework, project, track, internship, track, or exam option must complete 24 units of formal letter-graded coursework. Overview\n\n\n\nIn addition to Uni

In [54]:
from langchain_community.llms import GPT4All
llm = GPT4All(
    model="qwen2.5-coder-7b-instruct-q4_0.gguf",  # use a stable small local model
    allow_download=True
)

In [ ]:
prompt = f"""
You are a knowledgeable assistant. 
Please explain in depth, step by step, using detailed reasoning.

Context:a
{context}

Question: {query}
Answer:"""

In [40]:
prompt

"Answer the question accordingto the context provided, provide brief and correct answer \nContext: Additional Admission Requirements\n\n\n\n\n\nIndividual program admission requirements over and above admission to NAU are required.\n\n\n\nA Bachelor degree in Computer Science or a related field.\n\n\n\nComputer Science Milestone. Students must pass the Initial Skills Inventory Exam, or complete CS 500 and CS 501 before enrolling in graduate-level CS courses. See the department website for details.\n\n\n\nEssay/Letter of Intent/Personal Statement*\n\n\n\nRecommendation(s)/Reference(s)*\n\n\n\n*See the application for details.\n\nMaster's Requirements\n\n\n\n\n\nThis Master's degree requires 30 units distributed as follows: Computer Science Coursework:\xa03 units Non-Thesis or Thesis Option: 27 units\t Non-Thesis Option Thesis Option Thesis Option with Research  Take the following 30 units:\n\n\n\nStudents completing a non-thesis, coursework, project, track, internship, track, or exam op

In [48]:
response = llm.invoke(prompt, max_tokens=1000)     # make it more creative

In [50]:
print(response)

 The core requirements for the NAU Computer Science MS program include a Bachelor's degree in Computer Science or a related field, passing the Initial Skills Inventory Exam (or completing CS 500 and CS 501), an essay/letter of intent/personal statement, recommendation(s)/reference(s), and completing 30 units distributed as follows: 24 units of formal letter-graded coursework for non-thesis options.

To elaborate on the core requirements:

1. Bachelor's degree in Computer Science or a related field:
   - This requirement ensures that students have foundational knowledge in computer science before pursuing advanced studies.
   - A related field may include other disciplines such as software engineering, data science, or cybersecurity.

2. Initial Skills Inventory Exam (or completing CS 500 and CS 501):
   - The exam assesses the student's initial skills in computer science to ensure they are prepared for graduate-level coursework.
   - Alternatively, students can complete specific course

In [51]:
from langchain_community.vectorstores import FAISS
vectorstore_ = FAISS.load_local("vectorstore", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore_.as_retriever(kwargs={"k":6}) #fetches relevant documents
llm = OllamaLLM(model="llama3")

In [52]:
response = llm.invoke(prompt)
print(response)

To determine the core requirements for the NAU Computer Science MS program, let's break down the information provided step by step.

1. **Additional Admission Requirements**: The program requires individual program admission requirements over and above admission to NAU.
2. **Master's Requirements**: To complete a Master's degree in Computer Science, students need to take 30 units distributed as follows:
	* **Computer Science Coursework**: 3 units
	* **Non-Thesis or Thesis Option**: 27 units (24 units of formal letter-graded coursework for non-thesis option)
3. **Non-Thesis or Thesis Option**: The program offers two options: Non-Thesis and Thesis.
4. **Thesis, Oral Defense, and Research**: Depending on the chosen emphasis, thesis, oral defense, or research may be required.

Now, to identify the core requirements, let's focus on the minimum units needed for completion (30 units) and the specific coursework required:

* **Computer Science Coursework**: 3 units
* **Non-Thesis or Thesis Opt

In [8]:
#Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant that answers questions using the provided context." \
    "Question: {input}\n\n"
    "Context: {context}"
    "Provide a clear and concise answer based on the context above"
)

In [9]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [10]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
query = "What are the core requirements for the NAU Computer Science MS program?"
result = retrieval_chain.invoke({"input": query})
print("Answer:\n", result["answer"])

# Step 11: Display results
print("Answer:\n", result["answer"])
print("\nRetrieved context chunks:")
for i, doc in enumerate(result["context"]):
    print(f"--- Chunk {i+1} ---")
    print(doc.page_content[:300], "...\n")

Answer:
 According to the provided context, the core requirements for the NAU Computer Science MS program are:

* A Bachelor's degree in Computer Science or a related field.

These are the individual program admission requirements over and above admission to NAU.
Answer:
 According to the provided context, the core requirements for the NAU Computer Science MS program are:

* A Bachelor's degree in Computer Science or a related field.

These are the individual program admission requirements over and above admission to NAU.

Retrieved context chunks:
--- Chunk 1 ---
Additional Admission Requirements





Individual program admission requirements over and above admission to NAU are required.



A Bachelor degree in Computer Science or a related field. ...

--- Chunk 2 ---
of Science in Computer Science - Non-thesis at NAU.A total of 12 units will be shared between the plans selected from:400-level CS coursework (6 units)500-level CS coursework (6 units)Excluding CS ...

--- Chunk 3 ---
of

In [ ]:
from langchain_community.llms import GPT4All
llm = GPT4All(model="mistral-7b-instruct.Q4_0.gguf")

In [2]:
from langchain_classic.document_loaders import WebBaseLoader
from langchain_classic.embeddings import SentenceTransformerEmbeddings
from langchain_classic.vectorstores import FAISS
from gpt4all import GPT4All

d:\ML Projects\nau-smart-campus-assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 